# Phylogenetic Analysis

## A) COVID-19 COMPLETE GENOME (Greece: Athens)

### 1. Extracting the data

Data is extracted from [NCBI for VIRUS COVID-19](https://www.ncbi.nlm.nih.gov/labs/virus/vssi/#/virus?SeqType_s=Nucleotide&VirusLineage_ss=Severe%20acute%20respiratory%20syndrome%20coronavirus%202,%20taxid:2697049&Country_s=Greece). The downloaded data concernes Nucleotide (complete genome) of COVID-19 from different patients submitted from Athens, Greece. The accession numbers which are to be used for the fylogenetic analysis are
* MT459832
* MT459833
* MT459834
* MT459835
* MT459836

In [1]:
COVID_GREECE=open('sequences.fasta',"r")
#data=COVID_GREECE.read()
#print(data)

#construction of a matrix for the five different sequences
matrix = [[], [], [], [], []]
for x in range(5): 
    line = COVID_GREECE.readline()
    while True:
        char = COVID_GREECE.read(1)
        if char == '>' or char =='': #define the new seq
            break
        if (char !='\n' and char !='\r') :
            matrix[x].append(char)
            


### 2. Compute the distance between the sequences

*Assumptions *
* No coding parts are included
* Jukes-Cantor model: real distance is given by the form $d=-\frac{3}{4}ln\left(1-\frac{4}{3}D\right)$
* D is the observed distance D=$\frac{miss-matches}{length\quad of\quad the\quad sequence}$ 

In [2]:
import string

def observed_distance(matrix):
    length=len(matrix[1]) #the length of the sequence
    N=len(matrix)         #number of seq
    
    distance = [[0 for x in range(N)] for x in range(N)]

    for i in range(0,N):
        for j in range(i+1,N):
            mismatch=0
            #find the number of the different bases between the sequences
            for z in range(0,length):
                if(matrix[i][z]!=matrix[j][z]): #between the same seq the distance is 0
                    mismatch+=1
            #matrix construction of this form:
            # /   A          B        C       D         E
            # A   0        D(A,B)   D(A,C)   D(A,D)   D(A,E)
            # B   D(A,B)     0      D(B,C)   D(B,D)   D(B,E)
            # C   D(A,C)   D(B,C)     0      D(C,D)   D(C,E)
            # D   D(A,D)   D(B,D)   D(C,D)     0      D(D,E) 
            # E   D(A,E)   D(B,E)   D(C,E)   D(D,E)     0
            
            distance[i][j]=mismatch/length
            distance[j][i]=mismatch/length
            
    return distance

import numpy as np

def real_distance(obs_dist):
    N=len(obs_dist)
    
            #matrix construction of this form:
            # /   A          B        C       D         E
            # A   0        D(A,B)   D(A,C)   D(A,D)   D(A,E)
            # B   0          0      D(B,C)   D(B,D)   D(B,E)
            # C   0          0        0      D(C,D)   D(C,E)
            # D   0          0        0        0      D(D,E) 
            # E   0          0        0        0        0
            
    d = [[0 for x in range(N)] for x in range(N)]
    for i in range(0,N):
        for j in range(i+1,N):
            d[i][j]=-3/4*np.log(1-4/3*obs_dist[i][j])
            
    return d

In [3]:
obs_dist=observed_distance(matrix)       
real_dist=real_distance(obs_dist) 

As you can see there is not important difference between the real and the observed distance, since we have include no coding parts (complete genome) and also there wasn't enough time in the evolution history for this case.

In [4]:
real_dist

[[0,
  0.0007717430888536821,
  0.00023479427740956833,
  0.0004025494897042292,
  0.0004025494897042292],
 [0, 0, 0.0006039052866046445, 0.0011747071228148484, 0.0007717430888536821],
 [0, 0, 0, 0.0005703422328086179, 0.00023479427740956833],
 [0, 0, 0, 0, 0.0008053151568986336],
 [0, 0, 0, 0, 0]]

In [5]:
obs_dist

[[0,
  0.0007713461667449192,
  0.00023475752900932323,
  0.000402441478301697,
  0.000402441478301697],
 [0.0007713461667449192,
  0,
  0.0006036622174525455,
  0.001173787645046616,
  0.0007713461667449192],
 [0.00023475752900932323,
  0.0006036622174525455,
  0,
  0.0005701254275940707,
  0.00023475752900932323],
 [0.000402441478301697,
  0.001173787645046616,
  0.0005701254275940707,
  0,
  0.000804882956603394],
 [0.000402441478301697,
  0.0007713461667449192,
  0.00023475752900932323,
  0.000804882956603394,
  0]]

### 3. Estimate the Phylogenetic Tree
We use the UPGMA method:
 1. Find the minimum value for the distance D[i,j].
 
 2. Merge sequences i and j.
 
 3. Let x denote the node with wich i, j are conected. Setting L=D[i,j]/2 ensures that elements i and j are equidistant from x.
 
 4. Update the initial distances D. The new matrix has $(N-1)^2$ values and the elements i,j have been replaced by the node x. The new distance between the node x and the others sequences (s) is calculated as $D'[s,x]=\frac{D[i,s]+D[j,s]}{2}$.
 
 5. Repeat.

 

In [6]:
def UPGMA(dist,N):
    
    nodes = [0 for x in range(N-1)] #N-2 = number of nodes
    branch = [0 for x in range(N-1)] #branches = nodes+1
    
    #Function for the minimum value and the corresponding indeces
    def find_min(matrix,rows,columns):
        minimum = 100 #arbitanary large number
        minr,minc = 0,0
        for i in range(0,rows): #the last row has the names of the seq/nodes
            for j in range(0,columns):
                if matrix[i][j] < minimum and matrix[i][j] != 0.0 :
                    minimum = matrix[i][j] #minimum value
                    minr,minc = i,j        #indeces corresponding to the minimum value
        return minr,minc,minimum
    
    #Calculating the new matrix
    def new_matrix(matrix,N):

        #Step 1
        minr,minc,minimum = find_min(matrix,N-1,N)
        
        #Step 3
        node = '('+matrix[N-1][minr]+','+matrix[N-1][minc]+')' #node x from step 3
        L = minimum/2 #L from step 3
        
        #update the values: step 4
        for i in range(0,N-1):
            if matrix[i][minr] != 0 and matrix[i][minr] != minimum:
                new_val = (matrix[i][minr]+ matrix[i][minc])/2
                matrix[i][minr] = new_val

        #Step 2
        matrix[N-1][minr] = node #replace the element i with the node
        matrix[minr][minr] = 0
        
        #delete = replace value/name with zero
        #delete column
        for i in range(0,N-1):
            matrix[i][minc] = 0.0
        #delete rows
        if minc != N-1:
            for i in range(0,N):
                matrix[minc][i] = 0.0


        print (node)
        print ("Branch Length Estimation: ",L)
        print ("\n")
        
        return matrix,node,L

    #Step 5
    for n in range(0,N-1):
        matrix_new,node,L=new_matrix(dist,N)
        nodes[n] = node
        branch[n] = L
        
    return matrix_new,nodes,branch

In [7]:
N=len(real_dist)
for i in range(0,N):
        real_dist[N-1][i]=string.ascii_uppercase[i]

In [8]:
new_matrix,nodes,branch=UPGMA(real_dist,N)

(A,C)
Branch Length Estimation:  0.00011739713870478416


((A,C),D)
Branch Length Estimation:  0.0002012747448521146


(((A,C),D),E)
Branch Length Estimation:  0.0002012747448521146


((((A,C),D),E),B)
Branch Length Estimation:  0.00038587154442684104




In [9]:
nodes

['(A,C)', '((A,C),D)', '(((A,C),D),E)', '((((A,C),D),E),B)']

In [10]:
branch

[0.00011739713870478416,
 0.0002012747448521146,
 0.0002012747448521146,
 0.00038587154442684104]

### 4. Draw the Phylogenetic Tree

In [11]:
stracture=''+nodes[N-2]+';'

In [12]:
#!pip install ete3
from ete3 import Tree
t = Tree( stracture )

print(t)



            /-A
         /-|
      /-|   \-C
     |  |
   /-|   \-D
  |  |
--|   \-E
  |
   \-B


## B) Prostaglandin Synthesis

Data:
* [Bos Taurus (Cow)](https://www.ncbi.nlm.nih.gov/nuccore/198282106)
* [Ovis aries(Sheep)](https://www.ncbi.nlm.nih.gov/nuccore/57164168)
* [Rattus norvegicus (Rat)](https://www.ncbi.nlm.nih.gov/nuccore/415637)
* [Xenopus laevis(Frog)](https://www.ncbi.nlm.nih.gov/nuccore/117307525)
* [Mesochaetopterus taylori ((Marine worm)](https://www.ncbi.nlm.nih.gov/nuccore/933798131)
* [Sus scrofa (Wild pig)](https://www.ncbi.nlm.nih.gov/nuccore/AJ001201.1)

In [13]:
prostaglandin=open('prostaglandin.txt',"r")

#construction of a matrix for the five different sequences
matrix = [[], [], [], [], [], []]
for x in range(6): 
    line = prostaglandin.readline()
    while True:
        char = prostaglandin.read(1)
        if char == '>' or char =='': #define the new seq
            break
        if (char !='\n' and char !='\r') :
            matrix[x].append(char)
            

Step 2: Compute Distance 

In [14]:
obs_dist=observed_distance(matrix)       
real_dist=real_distance(obs_dist) 

Step 3: Estimate Phylogenetic Tree

In [15]:
N=len(real_dist)
real_dist[N-1][0]='Cow'
real_dist[N-1][1]='Sheep'
real_dist[N-1][2]='Worm'
real_dist[N-1][3]='Rat'
real_dist[N-1][4]='Frog'
real_dist[N-1][5]='Pig'

In [16]:
new_matrix,nodes,branch=UPGMA(real_dist,N)

(Cow,Sheep)
Branch Length Estimation:  0.014573783032793972


((Cow,Sheep),Pig)
Branch Length Estimation:  0.04722415892008058


(((Cow,Sheep),Pig),Rat)
Branch Length Estimation:  0.09545733130194908


((((Cow,Sheep),Pig),Rat),Frog)
Branch Length Estimation:  0.2276360969257573


(((((Cow,Sheep),Pig),Rat),Frog),Worm)
Branch Length Estimation:  0.5413867683955758




Step 4: Draw the Phylogenetic Tree

In [17]:
stracture=''+nodes[N-2]+';'
t = Tree( stracture )

print(t)



               /-Cow
            /-|
         /-|   \-Sheep
        |  |
      /-|   \-Pig
     |  |
   /-|   \-Rat
  |  |
--|   \-Frog
  |
   \-Worm
